In [2]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [3]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [4]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [5]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [6]:
ncs = allURL[0]
dtime2 = netCDF4.num2date(ncs[0:],units = "days since 1985-01-01 00:00:00 UTC", calendar='gregorian')
print dtime2

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: unorderable dtypes; returning scalar but in the future this will be an error
  


AttributeError: 'bool' object has no attribute 'all'

In [7]:
boxLat = []
boxLon = []
boxWh = []
boxT = []

In [8]:
#Cyclone coordinates for marcia
latmin = -24.5 
latmax = -23.0  
lonmin = 152.5 
lonmax = 155.5

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [9]:
# Marcia dates
# start_date = dt.datetime(1985,2,15)
# end_date = dt.datetime(2018,2,21)

start_date = dt.datetime(2010,1,1)
end_date = dt.datetime(2010,12,31)


In [10]:
for u in range (len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        
        timing = netCDF4.num2date(tt,time_var.units)
        
        
        for k in range(len(timing)):
            if timing[k] >= start_date and timing[k] <= end_date:
                if lats[k]>latmin and lats[k]<latmax and lons[k]>lonmin and lons[k]<lonmax:
                    boxLat.append(lats[k])
                    boxLon.append(lons[k])
                    boxWh.append(wh[k])
                    boxT.append(tt[k])
                    #print timing[k]
                    
#         id_2 = netCDF4.date2index(end_date,time_var,select='nearest')
#         diff3 = start_date-netCDF4.num2date(tt[id_2],time_var.units)
        
#         diff2 = end_date-netCDF4.num2date(tt[id_2],time_var.units)
        #dh1 = (diff1.seconds + diff1.days * 24 * 3600)/3600.
#         dh2 = (diff2.seconds + diff2.days * 24 * 3600)/3600. #diff in end date (actual vs nearest) in days
#         dh3 = (diff3.seconds + diff3.days * 24 * 3600)/3600. #diff btn start date and nearest end date in days

#         id_st = -1
#         id_et = -1
#         if dh3<0.:
#             id_st = netCDF4.date2index(start_date,time_var,select='after')
            
#         if dh2>-24.:
#             id_et = netCDF4.date2index(end_date,time_var,select='before')
            
#         if id_st<id_et:
#             print id_st,id_et,netCDF4.num2date(tt[id_st],time_var.units)
#             print netCDF4.num2date(tt[id_et],time_var.units)
#             # Check if satellite points are in the box
#             id1 = lats>latmin
#             id2 = lats<latmax
#             id3 = lons>lonmin
#             id4 = lons<lonmax
#             id1 = id1*1
#             id2 = id2*1
#             id3 = id3*1
#             id4 = id4*1
#             tot = id1+id2+id3+id4
            
#             # If this is the case then the sum should be 4
#             inside = np.where(tot==4)[0]
#             idtime = np.arange(id_st,id_et+1)

#             idin = np.intersect1d(inside, idtime)

#             if len(idin)>0:
#                 boxLat.append(lats[idin])
#                 boxLon.append(lons[idin])
#                 boxWh.append(wh[idin])
#                 boxT.append(tt[idin])

In [11]:
len(boxT)

4483

In [12]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])

In [14]:
print len(wh)
wh

4483


masked_array(data=[2.12000012, 2.17700005, 2.13600016, ..., 1.62700009,
                   0.97100008, 0.48500001],
             mask=False,
       fill_value=1e+20)

In [15]:
print 'records for Olga in CBG Box 2:', len(boxWh)

records for Olga in CBG Box 2: 4483


In [17]:
#wave height pos values

ids = np.where(wh>0)[0]
whpos = wh[ids]
len(whpos)

4462

In [18]:
print np.mean(whpos)

1.7246249139430825


In [19]:
print 'mean whpos', np.mean(whpos)
print 'max whpos',max(whpos)
print 'min whpos', min (whpos)

#try diff wave names
print 'mean wave height', np.mean(wh)

print 'min lon',min(lon)
print 'max lon',max(lon)

print 'min lat', min(lat)
print 'max lat',max(lat)

print min(netCDF4.num2date(tt,time_var.units))
print min (netCDF4.num2date(tt[ids],time_var.units))
print max (netCDF4.num2date(tt,time_var.units))
print max (netCDF4.num2date(tt[ids],time_var.units))

mean whpos 1.7246249139430825
max whpos 10.872000694274902
min whpos 0.024000000208616257
mean wave height 1.7165461445491934
min lon 152.50064
max lon 155.49367
min lat -24.499977
max lat -23.000303
2010-01-01 12:03:16.851558
2010-01-01 12:03:16.851558
2010-12-27 06:07:26.535156
2010-12-27 06:07:26.535156


In [20]:
for k in range(len(tt)):
    if wh[k]>0:
        print netCDF4.num2date(tt[k],time_var.units)

2010-01-09 19:22:31.671874
2010-01-09 19:22:32.703120
2010-01-09 19:22:33.718745
2010-01-09 19:22:34.734370
2010-01-09 19:22:35.749996
2010-01-09 19:22:36.781251
2010-01-09 19:22:37.796876
2010-01-09 19:22:38.812502
2010-01-09 19:22:39.843747
2010-01-09 19:22:40.859372
2010-01-09 19:22:41.874998
2010-01-09 19:22:42.890623
2010-01-09 19:22:43.921878
2010-01-09 19:22:44.937504
2010-01-19 17:21:03.765629
2010-01-19 17:21:04.796875
2010-01-19 17:21:05.812500
2010-01-19 17:21:06.828125
2010-01-19 17:21:07.843750
2010-01-19 17:21:08.874996
2010-01-19 17:21:09.890621
2010-01-19 17:21:10.906246
2010-01-19 17:21:11.937502
2010-01-19 17:21:12.953127
2010-01-19 17:21:13.968752
2010-01-19 17:21:14.984377
2010-01-19 17:21:16.015623
2010-01-19 17:21:17.031248
2010-01-19 17:21:18.046873
2010-01-29 15:19:35.718749
2010-01-29 15:19:36.734374
2010-01-29 15:19:37.749999
2010-01-29 15:19:39.796880
2010-01-29 15:19:40.812505
2010-01-29 15:19:41.843751
2010-01-29 15:19:42.859376
2010-01-29 15:19:43.875001
2

2010-12-03 00:34:21.512697
2010-12-07 10:10:02.406254
2010-12-12 22:32:44.734371
2010-12-12 22:32:45.765627
2010-12-12 22:32:46.781252
2010-12-12 22:32:47.796877
2010-12-12 22:32:48.828123
2010-12-12 22:32:49.843748
2010-12-12 22:32:50.859373
2010-12-12 22:32:51.874998
2010-12-12 22:32:52.906254
2010-12-12 22:32:53.921879
2010-12-17 08:08:34.250000
2010-12-22 20:31:15.921874
2010-12-22 20:31:16.937499
2010-12-22 20:31:17.953124
2010-12-22 20:31:18.968750
2010-12-22 20:31:19.999995
2010-12-22 20:31:21.015620
2010-12-22 20:31:22.031246
2010-12-22 20:31:23.062501
2010-12-22 20:31:24.078126
2010-12-22 20:31:25.093752
2010-01-04 07:00:11.359380
2010-01-04 07:00:12.382810
2010-01-04 07:00:13.398435
2010-01-04 07:00:14.421876
2010-01-04 07:00:15.437501
2010-01-04 07:00:16.460941
2010-01-04 07:00:17.484372
2010-01-04 07:00:18.499997
2010-01-04 07:00:19.523437
2010-01-04 07:00:20.539062
2010-01-04 07:00:21.562503
2010-01-04 07:00:22.578128
2010-01-04 07:00:23.601558
2010-01-04 07:00:24.624999
2

2010-10-18 20:17:30.328122
2010-10-18 20:17:31.343748
2010-10-18 20:17:32.359373
2010-10-18 20:17:33.374998
2010-10-18 20:17:34.406254
2010-10-18 20:17:35.421879
2010-10-18 20:17:36.437504
2010-10-18 20:17:37.468750
2010-10-18 20:17:38.484375
2010-10-18 20:17:39.500000
2010-10-18 20:17:40.515625
2010-10-18 20:17:41.546871
2010-10-18 20:17:42.562496
2010-10-18 20:17:43.578121
2010-10-18 20:17:44.609377
2010-10-24 08:39:58.296872
2010-10-24 08:39:59.320313
2010-10-24 08:40:00.335938
2010-10-24 08:40:01.359378
2010-10-24 08:40:02.375003
2010-10-24 08:40:03.398434
2010-10-24 08:40:04.414059
2010-10-28 18:15:55.437495
2010-10-28 18:15:56.453120
2010-10-28 18:15:57.484376
2010-10-28 18:15:58.500001
2010-10-28 18:15:59.515626
2010-10-28 18:16:00.546872
2010-10-28 18:16:01.562497
2010-10-28 18:16:02.578122
2010-10-28 18:16:03.593748
2010-10-28 18:16:04.625003
2010-10-28 18:16:05.640628
2010-10-28 18:16:06.656254
2010-10-28 18:16:07.687499
2010-10-28 18:16:08.703124
2010-10-28 18:16:09.718749
2

2010-07-29 02:07:28.429687
2010-07-29 02:07:29.373044
2010-08-27 00:42:15.837895
2010-08-27 00:42:16.781251
2010-08-27 00:42:17.724608
2010-08-27 00:42:18.667964
2010-08-27 00:42:19.611813
2010-08-27 00:42:20.555180
2010-08-27 00:42:21.498536
2010-08-27 00:42:22.441893
2010-08-27 00:42:23.385249
2010-08-27 00:42:24.328615
2010-08-27 00:42:25.272465
2010-08-27 00:42:26.215821
2010-08-27 00:42:27.159177
2010-08-27 00:42:28.102544
2010-08-27 00:42:29.045900
2010-08-27 00:42:29.989257
2010-08-27 00:42:30.933106
2010-09-22 23:19:43.593754
2010-09-22 23:19:44.531247
2010-09-22 23:19:45.468749
2010-09-22 23:19:46.421872
2010-09-22 23:19:47.359375
2010-09-22 23:19:48.296877
2010-09-22 23:19:49.250000
2010-09-22 23:19:50.187503
2010-09-22 23:19:51.140626
2010-09-22 23:19:52.078128
2010-09-22 23:19:53.015620
2010-09-22 23:19:53.968754
2010-09-22 23:19:54.906246
2010-09-22 23:19:55.843748
2010-09-22 23:19:56.796871
2010-09-22 23:19:57.734374
2010-09-22 23:19:58.687497
2010-09-22 23:19:59.624999
2

2010-08-31 11:57:32.125002
2010-08-31 11:57:33.242186
2010-08-31 11:57:34.351564
2010-08-31 11:57:35.468748
2010-08-31 11:57:36.585942
2010-08-31 11:57:37.695310
2010-09-12 23:27:57.031251
2010-09-12 23:27:58.140629
2010-09-12 23:27:59.265628
2010-09-12 23:28:00.374997
2010-09-12 23:28:01.484375
2010-09-12 23:28:02.593754
2010-09-12 23:28:03.718752
2010-09-12 23:28:04.828121
2010-09-28 23:25:02.390627
2010-09-28 23:25:03.499996
2010-09-28 23:25:04.625004
2010-09-28 23:25:05.734373
2010-09-28 23:25:07.968750
2010-10-05 11:57:25.367184
2010-10-05 11:57:26.484377
2010-10-05 11:57:27.601561
2010-10-05 11:57:28.710939
2010-10-05 11:57:29.828123
2010-10-05 11:57:30.937501
2010-10-05 11:57:32.054685
2010-10-05 11:57:33.171879
2010-10-17 23:27:50.375001
2010-10-17 23:27:51.500000
2010-10-17 23:27:52.609378
2010-10-17 23:27:53.718746
2010-10-17 23:27:54.828125
2010-10-17 23:27:55.953124
2010-10-17 23:27:57.062502
2010-10-17 23:27:58.171870
2010-11-04 11:56:16.359370
2010-11-04 11:56:17.468749
2

2010-09-28 23:24:57.937503
2010-09-28 23:24:59.046871
2010-09-28 23:25:00.171870
2010-09-28 23:25:01.281249
2010-10-05 11:57:34.281247
2010-10-05 11:57:35.398441
2010-10-05 11:57:36.507809
2010-10-05 11:57:37.625003
2010-10-05 11:57:38.742186
2010-10-05 11:57:39.851565
2010-10-05 11:57:40.968748
2010-10-05 11:57:42.078127
2010-10-05 11:57:43.195311
2010-10-05 11:57:44.304689
2010-10-05 11:57:45.421873
2010-10-05 11:57:46.539066
2010-10-05 11:57:47.648435
2010-10-05 11:57:48.765628
2010-10-05 11:57:49.874997
2010-10-17 23:27:33.671873
2010-10-17 23:27:34.781251
2010-10-17 23:27:37.015628
2010-10-17 23:27:38.124997
2010-10-17 23:27:39.234375
2010-10-17 23:27:40.359374
2010-10-17 23:27:41.468752
2010-10-17 23:27:42.578121
2010-10-17 23:27:43.687499
2010-10-17 23:27:44.812498
2010-10-17 23:27:45.921876
2010-10-17 23:27:47.031255
2010-10-17 23:27:48.156254
2010-10-17 23:27:49.265622
2010-10-21 11:54:44.906245
2010-10-21 11:54:46.023439
2010-10-21 11:54:47.140623
2010-10-21 11:54:48.250001
2

In [21]:
tsort = np.sort(tt)
for k in range(len(tsort)):
    print netCDF4.num2date(tsort[k],time_var.units)

2010-01-01 12:03:16.851558
2010-01-01 12:03:17.960936
2010-01-01 12:03:19.078130
2010-01-01 12:03:20.195313
2010-01-01 12:03:21.304692
2010-01-01 12:03:22.421875
2010-01-01 12:03:23.531254
2010-01-01 12:03:24.648437
2010-01-01 12:03:26.874999
2010-01-01 12:03:27.992183
2010-01-01 12:03:29.101561
2010-01-01 12:03:30.218745
2010-01-01 12:03:31.328124
2010-01-01 12:03:32.445317
2010-01-01 12:03:33.562501
2010-01-01 12:03:34.671879
2010-01-01 12:03:35.789063
2010-01-01 12:03:36.898441
2010-01-01 12:03:38.015625
2010-01-01 12:03:39.132809
2010-01-01 12:03:40.242187
2010-01-01 12:03:41.359371
2010-01-04 07:00:01.156247
2010-01-04 07:00:02.179687
2010-01-04 07:00:03.195312
2010-01-04 07:00:04.218753
2010-01-04 07:00:05.242183
2010-01-04 07:00:06.257808
2010-01-04 07:00:07.281249
2010-01-04 07:00:08.296874
2010-01-04 07:00:09.320314
2010-01-04 07:00:10.343755
2010-01-04 07:00:11.359380
2010-01-04 07:00:12.382810
2010-01-04 07:00:13.398435
2010-01-04 07:00:14.421876
2010-01-04 07:00:15.437501
2

2010-03-25 11:54:39.070308
2010-03-25 11:54:40.187502
2010-03-25 11:54:41.296870
2010-03-25 11:54:42.414064
2010-03-25 11:54:43.523432
2010-03-25 11:54:44.640626
2010-03-25 11:54:45.757810
2010-03-25 11:54:46.867188
2010-03-25 11:54:47.984372
2010-03-25 11:54:49.093750
2010-03-25 11:54:50.210934
2010-03-25 11:54:51.328128
2010-03-25 11:54:52.437496
2010-03-25 11:54:53.554690
2010-03-25 11:54:54.664058
2010-03-25 11:54:55.781252
2010-03-25 11:54:56.898436
2010-03-25 11:54:58.007814
2010-03-25 11:54:59.124998
2010-03-25 11:55:00.234376
2010-03-25 11:55:01.351560
2010-03-25 11:55:02.468753
2010-03-25 11:55:03.578122
2010-03-28 12:00:24.757812
2010-03-28 12:00:25.867190
2010-03-28 12:00:26.984374
2010-03-28 12:00:28.093752
2010-03-28 12:00:29.210936
2010-03-28 12:00:30.320314
2010-03-28 12:00:31.437498
2010-03-28 12:00:32.554692
2010-03-28 12:00:33.664060
2010-03-28 12:00:34.781254
2010-03-28 12:00:35.890622
2010-03-28 12:00:37.007816
2010-03-28 12:00:38.124999
2010-03-28 12:00:40.351561
2

2010-06-11 22:36:43.109371
2010-06-11 22:36:44.124996
2010-06-11 22:36:45.156251
2010-06-11 22:36:46.171876
2010-06-11 22:36:47.187502
2010-06-11 22:36:48.203127
2010-06-11 22:36:49.234372
2010-06-11 22:36:50.249998
2010-06-11 22:36:51.265623
2010-06-11 22:36:52.296879
2010-06-11 22:36:53.312504
2010-06-11 22:36:54.328129
2010-06-11 22:36:55.343754
2010-06-11 22:36:56.375000
2010-06-15 23:24:47.937497
2010-06-15 23:24:49.046875
2010-06-15 23:24:50.156254
2010-06-15 23:24:51.281253
2010-06-15 23:24:52.390621
2010-06-15 23:24:53.500000
2010-06-15 23:24:54.624998
2010-06-15 23:24:55.734377
2010-06-15 23:24:56.843745
2010-06-15 23:24:57.968754
2010-06-15 23:24:59.078122
2010-06-15 23:25:00.187501
2010-06-15 23:25:01.296879
2010-06-15 23:25:02.421878
2010-06-15 23:25:03.531247
2010-06-15 23:25:04.640625
2010-06-15 23:25:05.765624
2010-06-15 23:25:06.875002
2010-06-15 23:25:07.984371
2010-06-15 23:25:09.093749
2010-06-15 23:25:10.218748
2010-06-15 23:25:11.328126
2010-06-17 10:58:56.249997
2

2010-08-24 23:24:55.984376
2010-08-24 23:24:57.093754
2010-08-24 23:24:58.218753
2010-08-24 23:24:59.328121
2010-08-24 23:25:00.437500
2010-08-24 23:25:01.546878
2010-08-24 23:25:02.671877
2010-08-24 23:25:03.781245
2010-08-24 23:25:04.890624
2010-08-24 23:25:06.015623
2010-08-24 23:25:07.125001
2010-08-24 23:25:08.234379
2010-08-24 23:25:09.359378
2010-08-24 23:25:10.468747
2010-08-24 23:25:11.578125
2010-08-24 23:25:12.687504
2010-08-25 00:44:41.799807
2010-08-25 00:44:42.743656
2010-08-25 00:44:43.687013
2010-08-25 00:44:44.630369
2010-08-25 00:44:45.573736
2010-08-25 00:44:46.517092
2010-08-25 00:44:47.460448
2010-08-25 00:44:48.404298
2010-08-25 00:44:49.347654
2010-08-25 00:44:50.291020
2010-08-25 00:44:51.234377
2010-08-25 00:44:52.177733
2010-08-25 00:44:53.121090
2010-08-25 00:44:54.064939
2010-08-25 00:44:55.008305
2010-08-25 00:44:55.951662
2010-08-25 00:44:56.895018
2010-08-25 00:44:57.838374
2010-08-25 00:44:58.781741
2010-08-25 00:44:59.725097
2010-08-25 00:45:00.668946
2

2010-10-24 08:40:01.359378
2010-10-24 08:40:02.375003
2010-10-24 08:40:03.398434
2010-10-24 08:40:04.414059
2010-10-24 08:40:05.437499
2010-10-24 08:40:06.460940
2010-10-24 08:40:07.476565
2010-10-24 08:40:08.499995
2010-10-24 08:40:09.515620
2010-10-24 08:40:10.539061
2010-10-24 08:40:11.562501
2010-10-24 08:40:12.578126
2010-10-24 08:40:13.601567
2010-10-24 08:40:14.617192
2010-10-25 21:49:52.484372
2010-10-25 21:49:53.421874
2010-10-25 21:49:54.374998
2010-10-25 21:49:55.312500
2010-10-25 21:49:56.250002
2010-10-25 21:49:57.203126
2010-10-25 21:49:58.140628
2010-10-25 21:49:59.078120
2010-10-25 21:50:00.031253
2010-10-25 21:50:00.968746
2010-10-25 21:50:01.921879
2010-10-25 21:50:02.859371
2010-10-25 21:50:03.796874
2010-10-25 21:50:04.749996
2010-10-25 21:50:05.687499
2010-10-25 21:50:06.625002
2010-10-25 21:50:07.578124
2010-10-25 21:50:08.515627
2010-10-25 21:50:09.468750
2010-10-25 21:50:10.406252
2010-10-25 21:50:11.343755
2010-10-25 21:50:12.296878
2010-10-25 21:50:13.234370
2

2010-12-22 20:31:25.093752
2010-12-23 12:00:55.999998
2010-12-23 12:00:57.117192
2010-12-23 12:00:58.226560
2010-12-23 12:00:59.343754
2010-12-23 12:01:00.453122
2010-12-23 12:01:01.570316
2010-12-23 12:01:02.687499
2010-12-23 12:01:03.796878
2010-12-23 12:01:04.914062
2010-12-23 12:01:06.023440
2010-12-23 12:01:07.140624
2010-12-23 12:01:08.257817
2010-12-23 12:01:09.367186
2010-12-23 12:01:10.484379
2010-12-23 12:01:11.593748
2010-12-23 12:01:12.710941
2010-12-23 12:01:13.828125
2010-12-23 12:01:14.937504
2010-12-23 12:01:16.054687
2010-12-23 12:01:17.164066
2010-12-23 12:01:18.281249
2010-12-23 12:01:19.398433
2010-12-23 12:01:20.507811
2010-12-25 23:32:28.093752
2010-12-25 23:32:29.203120
2010-12-25 23:32:30.328129
2010-12-25 23:32:31.437498
2010-12-25 23:32:32.546876
2010-12-25 23:32:33.671875
2010-12-25 23:32:34.781253
2010-12-25 23:32:35.890622
2010-12-25 23:32:37.015621
2010-12-25 23:32:38.124999
2010-12-25 23:32:39.234377
2010-12-25 23:32:40.343746
2010-12-25 23:32:41.468755
2